In [1]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [2]:
list=[[1,12,20200102,1,2,3,4,5]]

df=spark.createDataFrame(list,['code','part_num','start_date','daily_qty_1','daily_qty_2','daily_qty_3','daily_qty_4','daily_qty_5'])
df.show()

+----+--------+----------+-----------+-----------+-----------+-----------+-----------+
code|part_num|start_date|daily_qty_1|daily_qty_2|daily_qty_3|daily_qty_4|daily_qty_5|
+----+--------+----------+-----------+-----------+-----------+-----------+-----------+
 1| 12| 20200102| 1| 2| 3| 4| 5|
+----+--------+----------+-----------+-----------+-----------+-----------+-----------+

In [3]:
from pyspark.sql import functions as F
df1=df.withColumn("daily_qty", F.split(F.concat_ws(',',*(x for x in df.columns if x.startswith('daily_qty'))),','))\
.withColumn("size", F.size("daily_qty"))\
.withColumn("date", F.expr("""sequence(start_date,start_date+bigint(size)-1,1)""")).drop("size")\
.withColumn("temp", F.explode(F.arrays_zip("date", "daily_qty")))\
.select("code","part_num","temp.date","temp.daily_qty")

In [4]:
df1.show()

+----+--------+--------+---------+
code|part_num| date|daily_qty|
+----+--------+--------+---------+
 1| 12|20200102| 1|
 1| 12|20200103| 2|
 1| 12|20200104| 3|
 1| 12|20200105| 4|
 1| 12|20200106| 5|
+----+--------+--------+---------+

In [5]:
int(df1.agg(F.max(F.col("daily_qty")).alias("max")).collect()[0][0])

Out[206]: 5

In [6]:
df = spark.createDataFrame([('what is the movie that features Tom Cruise','actor_movies','(movie|film).*(feature)|(in|on).*(movie|film)'),
    ('what is the movie that features Tom Cruise','artist_song','(who|what).*(sing|sang|perform)'),
    ('who is the singer for hotel califonia?','artist_song','(who|what).*(sing|sang|perform)')],  
['query','question_type','regex_patt'])
df.show(truncate=False)

+------------------------------------------+-------------+---------------------------------------------+
query |question_type|regex_patt |
+------------------------------------------+-------------+---------------------------------------------+
what is the movie that features Tom Cruise|actor_movies |(movie|film).*(feature)|(in|on).*(movie|film)|
what is the movie that features Tom Cruise|artist_song |(who|what).*(sing|sang|perform) |
who is the singer for hotel califonia? |artist_song |(who|what).*(sing|sang|perform) |
+------------------------------------------+-------------+---------------------------------------------+

In [7]:
df.filter("""query.rlike(regex_patt))""").show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o5987.filter.
: org.apache.spark.sql.catalyst.parser.ParseException: 
extraneous input ')' expecting <EOF>(line 1, pos 23)

== SQL ==
query.rlike(regex_patt))
-----------------------^^^

	at org.apache.spark.sql.catalyst.parser.ParseException.withCommand(ParseDriver.scala:241)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parse(ParseDriver.scala:117)
	at org.apache.spark.sql.execution.SparkSqlParser.parse(SparkSqlParser.scala:55)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parseExpression(ParseDriver.scala:44)
	at com.databricks.sql.parser.DatabricksSqlParser.parseExpression(DatabricksSqlParser.scala:46)
	at org.apache.spark.sql.Dataset.filter(Dataset.scala:1526)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 ParseException Traceback (most recent call last)
 <command-489342895924416> in <module> 
 ----> 1 df . filter ( "query.rlike(regex_patt))" ) . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in filter (self, condition) 
 1385 """
 1386 if isinstance ( condition , basestring ) : 
 -> 1387 jdf = self . _jdf . filter ( condition ) 
 1388 elif isinstance ( condition , Column ) : 
 1389 jdf = self . _jdf . filter ( condition . _jc ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 71 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 72 if s . startswith ( 'org.apache.spark.sql.catalyst.parser.ParseException: ' ) : 
 ---> 73 raise ParseException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 74 if s . startswith ( 'org.apache.spark.sql.streaming.StreamingQueryException: ' ) : 
 75 raise StreamingQueryException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 

 ParseException : "\nextraneous input ')' expecting <EOF>(line 1, pos 23)\n\n== SQL ==\nquery.rlike(regex_patt))\n-----------------------^^^\n"

In [8]:
df.withColumn("query1", F.expr("""regexp_extract(query, regex_patt)""")).show(truncate=False)

+------------------------------------------+-------------+---------------------------------------------+------+
query |question_type|regex_patt |query1|
+------------------------------------------+-------------+---------------------------------------------+------+
what is the movie that features Tom Cruise|actor_movies |(movie|film).*(feature)|(in|on).*(movie|film)|movie |
what is the movie that features Tom Cruise|artist_song |(who|what).*(sing|sang|perform) | |
who is the singer for hotel califonia? |artist_song |(who|what).*(sing|sang|perform) |who |
+------------------------------------------+-------------+---------------------------------------------+------+

In [9]:
df.withColumn("query1", F.expr("""regexp_extract(query, regex_patt)""")).show(truncate=False)

+------------------------------------------+-------------+---------------------------------------------+------+
query |question_type|regex_patt |query1|
+------------------------------------------+-------------+---------------------------------------------+------+
what is the movie that features Tom Cruise|actor_movies |(movie|film).*(feature)|(in|on).*(movie|film)|movie |
what is the movie that features Tom Cruise|artist_song |(who|what).*(sing|sang|perform) | |
who is the singer for hotel califonia? |artist_song |(who|what).*(sing|sang|perform) |who |
+------------------------------------------+-------------+---------------------------------------------+------+

In [10]:
list=[[1,12,20200102,1,2,3,4,'5']]

df=spark.createDataFrame(list,['code','part_num','start_date','daily_qty_1','daily_qty_2','daily_qty_3','daily_qty_4','daily_qty_5'])
df.show()

+----+--------+----------+-----------+-----------+-----------+-----------+-----------+
code|part_num|start_date|daily_qty_1|daily_qty_2|daily_qty_3|daily_qty_4|daily_qty_5|
+----+--------+----------+-----------+-----------+-----------+-----------+-----------+
 1| 12| 20200102| 1| 2| 3| 4| 5|
+----+--------+----------+-----------+-----------+-----------+-----------+-----------+

In [12]:
def dropstrcols(df):
    l=[]
    for i in range(len(df.columns)):
       if df.dtypes[i][1] =='string':
          l.append(df.columns[i])
          df1=df.drop(*(l))
          
          
    return df1
  
  
dropstrcols(df).show()
        
        
    
   
        
    

+----+--------+----------+-----------+-----------+-----------+-----------+
code|part_num|start_date|daily_qty_1|daily_qty_2|daily_qty_3|daily_qty_4|
+----+--------+----------+-----------+-----------+-----------+-----------+
 1| 12| 20200102| 1| 2| 3| 4|
+----+--------+----------+-----------+-----------+-----------+-----------+

In [13]:
testJson = [('{"drivernumber":1, "speed" : ["30.5", "40", "50", "25.25"]}',),
            ('{"drivernumber":2, "speed" : ["25.25", "10.11", "11", "25.25"]}',),
            ('{"drivernumber":3, "speed" : ["40", "50", "80", "42"]}',)
           ]


df=spark.createDataFrame(testJson,['col1'])
df.show(truncate=False)

+---------------------------------------------------------------+
col1 |
+---------------------------------------------------------------+
{"drivernumber":1, "speed" : ["30.5", "40", "50", "25.25"]} |
{"drivernumber":2, "speed" : ["25.25", "10.11", "11", "25.25"]}|
{"drivernumber":3, "speed" : ["40", "50", "80", "42"]} |
+---------------------------------------------------------------+

In [14]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
readSchema = StructType([
StructField("drivernumber", IntegerType(), True), 
StructField("speed", ArrayType(StringType(), True), True)])
df1=df.withColumn("col1", F.from_json("col1",readSchema)).select("col1.speed","col1.drivernumber")

In [15]:
df1.withColumn("speed", F.expr("""transform(speed, x-> float(x))""")).printSchema()

root
-- speed: array (nullable = true)
 |-- element: float (containsNull = true)
-- drivernumber: integer (nullable = true)

In [16]:
df1.withColumn("speed", F.col("speed").cast(ArrayType(FloatType(),True))).printSchema()

root
-- speed: array (nullable = true)
 |-- element: float (containsNull = true)
-- drivernumber: integer (nullable = true)

In [17]:
list=[[0,'Hi I heard about',['h', 'e', 'l', 'l', 'o'],['h e l', 'e l l']]]
df=spark.createDataFrame(list, ['id','words','tokens','ngrams'])
df.show()

+---+----------------+---------------+--------------+
 id| words| tokens| ngrams|
+---+----------------+---------------+--------------+
 0|Hi I heard about|[h, e, l, l, o]|[h e l, e l l]|
+---+----------------+---------------+--------------+

In [18]:
from pyspark.sql import functions as F
df.withColumn("ngrams", F.expr("""transform(ngrams,x-> regexp_replace(x,"\ ",""))""")).show()

+---+----------------+---------------+----------+
 id| words| tokens| ngrams|
+---+----------------+---------------+----------+
 0|Hi I heard about|[h, e, l, l, o]|[hel, ell]|
+---+----------------+---------------+----------+

In [19]:
from pyspark.sql import Row

df = spark.createDataFrame([Row(inputTokens=["a", "b", "c", "d", "e"])])
df.printSchema()

root
-- inputTokens: array (nullable = true)
 |-- element: string (containsNull = true)

In [20]:
from pyspark.ml.feature import NGram 

df = spark.createDataFrame([Row(inputTokens=["a", "b", "c", "d", "e"])])
ngram = NGram(n=2, inputCol="inputTokens", outputCol="nGrams")
df1=ngram.transform(df)

In [21]:
df1.printSchema()

root
-- inputTokens: array (nullable = true)
 |-- element: string (containsNull = true)
-- nGrams: array (nullable = true)
 |-- element: string (containsNull = false)

In [22]:
%scala
val df=Seq(
       (1,-7.1732833,32.0414966),
       (2,-7.1732844,32.0414406),
       (3,-7.1732833,32.0414966),
       (4,-7.1732833,32.0414966),
       (5,-7.1732833,32.0414966),
       (6,-7.1732833,32.0414966)
        ).toDF("seq","longitude","latitude")

df.show()

+---+----------+----------+
seq| longitude| latitude|
+---+----------+----------+
 1|-7.1732833|32.0414966|
 2|-7.1732844|32.0414406|
 3|-7.1732833|32.0414966|
 4|-7.1732833|32.0414966|
 5|-7.1732833|32.0414966|
 6|-7.1732833|32.0414966|
+---+----------+----------+

df: org.apache.spark.sql.DataFrame = [seq: int, longitude: double ... 1 more field]

In [23]:
%scala
import org.apache.spark.sql.functions.col

val max = df.agg(org.apache.spark.sql.functions.max(col("seq"))).collect()(0)(0).asInstanceOf[Int]

import org.apache.spark.sql.functions.col
max: Int = 6

In [24]:
%scala
import org.apache.spark.sql.functions.col

val max = df.agg(org.apache.spark.sql.functions.max(col("seq"))).first()

import org.apache.spark.sql.functions.col
max: org.apache.spark.sql.Row = [6]

In [25]:
%scala
import org.apache.spark.sql.functions.col

val max = df.agg(org.apache.spark.sql.functions.max(col("seq"))).collect()(0)(0).asInstanceOf[Int]

max: org.apache.spark.sql.DataFrame = [seq: int, longitude: double ... 2 more fields]

In [26]:
%scala
import org.apache.spark.sql.functions._
df.withColumn("max", lit(df.agg(%scala
import org.apache.spark.sql.functions._
df.withColumn("max", lit(df.agg(org.apache.spark.sql.functions.max($"id")).as[Int].first))max($"seq")).as[Int].first))

In [27]:
%scala
import org.apache.spark.sql.functions._
df.withColumn("max", lit(df.agg(org.apache.spark.sql.functions.max($"seq")).as[Int].first)).show(2)

+---+----------+----------+---+
seq| longitude| latitude|max|
+---+----------+----------+---+
 1|-7.1732833|32.0414966| 6|
 2|-7.1732844|32.0414406| 6|
+---+----------+----------+---+
only showing top 2 rows

import org.apache.spark.sql.functions._

In [28]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [29]:
list=[[1,[{'start' :1 , 'end' :2, 'flag' : 'A'},{'start' :3 , 'end' :5, 'flag' : 'S'}]],
      [2,[{'start' :1 , 'end' :5, 'flag' : 'A'}]]]

cschema = StructType([StructField("Index", StringType()),
                     StructField("flagArray",ArrayType(StructType([StructField("start", IntegerType()),
                StructField("end", IntegerType()),
                StructField("flag", StringType())])))])
df=spark.createDataFrame(list, schema=cschema)


In [30]:
df.selectExpr("*", "flatten(transform(flagArray, x -> array_repeat(x.flag, x.end-x.start+1))) as flag2").show()

+-----+--------------------+---------------+
Index| flagArray| flag2|
+-----+--------------------+---------------+
 1|[[1, 2, A], [3, 5...|[A, A, S, S, S]|
 2| [[1, 5, A]]|[A, A, A, A, A]|
+-----+--------------------+---------------+

In [31]:
display(df1)

Index,flagArray,flag2
1,"List(List(1, 2, A), List(3, 5, S))","List(A, A, S, S, S)"
2,"List(List(1, 5, A))","List(A, A, A, A, A)"


In [32]:
list=[[1,"[{start :1 , end :2, flag : A},{start :3 , end :5, flag : S}]"],
      [2,"[{start :1 , end :5, flag : A}]"]]
df=spark.createDataFrame(list, schema=cschema)
df.show(truncate=False)

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-2450092443183979> in <module> 
 1 list=[[1,"[{start :1 , end :2, flag : A},{start :3 , end :5, flag : S}]"],
 2 [2,"[{start :1 , end :5, flag : A}]"]]
 ----> 3 df = spark . createDataFrame ( list , schema = cschema ) 
 4 df . show ( truncate = False ) 

 /databricks/spark/python/pyspark/sql/session.py in createDataFrame (self, data, schema, samplingRatio, verifySchema) 
 815 rdd , schema = self . _createFromRDD ( data . map ( prepare ) , schema , samplingRatio ) 
 816 else : 
 --> 817 rdd , schema = self . _createFromLocal ( map ( prepare , data ) , schema ) 
 818 jrdd = self . _jvm . SerDeUtil . toJavaArray ( rdd . _to_java_object_rdd ( ) ) 
 819 jdf = self . _jsparkSession . applySchemaToPythonRDD ( jrdd . rdd ( ) , schema . json ( ) ) 

 /databricks/spark/python/pyspark/sql/session.py in _createFromLocal (self, data, schema) 
 440 write temp files . 
 441 """
 --> 442 data , schema = self . _wrap_data_schema ( data , schema ) 
 443 return self . _sc . parallelize ( data ) , schema
 444 

 /databricks/spark/python/pyspark/sql/session.py in _wrap_data_schema (self, data, schema) 
 416 # make sure data could consumed multiple times 
 417 if not isinstance ( data , list ) : 
 --> 418 data = list ( data ) 
 419 
 420 if schema is None or isinstance ( schema , ( list , tuple ) ) : 

 /databricks/spark/python/pyspark/sql/session.py in prepare (obj) 
 790 
 791 def prepare ( obj ) : 
 --> 792 verify_func ( obj ) 
 793 return obj
 794 elif isinstance ( schema , DataType ) : 

 /databricks/spark/python/pyspark/sql/types.py in verify (obj) 
 1387 def verify ( obj ) : 
 1388 if not verify_nullability ( obj ) : 
 -> 1389 verify_value ( obj ) 
 1390 
 1391 return verify

 /databricks/spark/python/pyspark/sql/types.py in verify_struct (obj) 
 1368 "length of fields (%d)" % (len(obj), len(verifiers))))
 1369 for v , ( _ , verifier ) in zip ( obj , verifiers ) : 
 -> 1370 verifier ( v ) 
 1371 elif hasattr ( obj , "__dict__" ) : 
 1372 d = obj . __dict__

 /databricks/spark/python/pyspark/sql/types.py in verify (obj) 
 1387 def verify ( obj ) : 
 1388 if not verify_nullability ( obj ) : 
 -> 1389 verify_value ( obj ) 
 1390 
 1391 return verify

 /databricks/spark/python/pyspark/sql/types.py in verify_array (obj) 
 1326 def verify_array ( obj ) : 
 1327 assert_acceptable_types ( obj ) 
 -> 1328 verify_acceptable_types ( obj ) 
 1329 for i in obj : 
 1330 element_verifier ( i ) 

 /databricks/spark/python/pyspark/sql/types.py in verify_acceptable_types (obj) 
 1276 if type ( obj ) not in _acceptable_types [ _type ] : 
 1277 raise TypeError(new_msg("%s can not accept object %r in type %s"
 -> 1278 % (dataType, obj, type(obj))))
 1279 
 1280 if isinstance ( dataType , StringType ) : 

 TypeError : field flagArray: ArrayType(StructType(List(StructField(start,LongType,true),StructField(end,LongType,true),StructField(flag,StringType,true))),true) can not accept object '[{start :1 , end :2, flag : A},{start :3 , end :5, flag : S}]' in type <class 'str'>

In [33]:
df.printSchema()

root
-- Index: long (nullable = true)
-- flagArray: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- start: long (nullable = true)
 | |-- end: long (nullable = true)
 | |-- flag: string (nullable = true)

In [34]:
display(df)

Index,flagArray
1,"List(List(1, 2, A), List(3, 5, S))"
2,"List(List(1, 5, A))"


In [35]:
cschema2=ArrayType(StructType([StructField("start", LongType()),
                StructField("end", LongType()),
                StructField("flag", StringType())]))
df.withColumn("flagArray1", F.from_json("flagArray",schema=cschema2)).select("flagArray1.*").show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o1672.withColumn.
: org.apache.spark.sql.AnalysisException: cannot resolve 'jsontostructs(`flagArray`)' due to data type mismatch: argument 1 requires string type, however, '`flagArray`' is of array<struct<start:bigint,end:bigint,flag:string>> type.;;
'Project [Index#109L, flagArray#110, jsontostructs(ArrayType(StructType(StructField(start,LongType,true), StructField(end,LongType,true), StructField(flag,StringType,true)),true), flagArray#110, Some(Etc/UTC)) AS flagArray1#122]
+- LogicalRDD [Index#109L, flagArray#110], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:126)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:302)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:105)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:117)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1$2.apply(QueryPlan.scala:122)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.immutable.List.map(List.scala:296)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:122)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$4.apply(QueryPlan.scala:127)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.mapExpressions(QueryPlan.scala:127)
	at org.apache.spark.sql.catalyst.plan

In [36]:
df.select("flagArray.*")

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o868.select.
: org.apache.spark.sql.AnalysisException: Can only star expand struct data types. Attribute: `ArrayBuffer(flagArray)`;
	at org.apache.spark.sql.catalyst.analysis.UnresolvedStar.expand(unresolved.scala:321)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveReferences$$buildExpandedProjectList$1.apply(Analyzer.scala:1027)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveReferences$$buildExpandedProjectList$1.apply(Analyzer.scala:1025)
	at scala.collection.TraversableLike$$anonfun$flatMap$1.apply(TraversableLike.scala:241)
	at scala.collection.TraversableLike$$anonfun$flatMap$1.apply(TraversableLike.scala:241)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at scala.collection.TraversableLike$class.flatMap(TraversableLike.scala:241)
	at scala.collection.AbstractTraversable.flatMap(Traversable.scala:104)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$.org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveReferences$$buildExpandedProjectList(Analyzer.scala:1025)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$apply$9.applyOrElse(Analyzer.scala:935)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$apply$9.applyOrElse(Analyzer.scala:930)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1$$anonfun$apply$1.apply(AnalysisHelper.scala:90)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1$$anonfun$apply$1.apply(AnalysisHelper.scala:90)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1.apply(AnalysisHelper.scala:89)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1.apply(AnalysisHelper.scala:86)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:194)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.resolveOperatorsUp(AnalysisHelper.scala:86)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsUp(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$.apply(Analyzer.scala:930)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$.apply(Analyzer.scala:775)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1$$anonfun$apply$1.apply(RuleExecutor.scala:112)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1$$anonfun$apply$1.apply(RuleExecutor.scala:109)
	at scala.collection.LinearSeqOptimized$class.foldLeft(LinearSeqOptimized.scala:124)
	at scala.collection.immutable.List.foldLeft(List.scala:84)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1.apply(RuleExecutor.scala:109)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1.apply(RuleExecutor.scala:101)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.execute(RuleExecutor.scala:101)
	at org.apach

In [37]:
import pandas as pd
from datetime import datetime
as_of_date = datetime.strptime('2013-01-01', '%Y-%m-%d')
columns = ['id', 'row', 'month']
vals = [('A', 1, as_of_date)]
df = spark.createDataFrame(vals, columns)
df.show()

+---+---+-------------------+
 id|row| month|
+---+---+-------------------+
 A| 1|2013-01-01 00:00:00|
+---+---+-------------------+

In [38]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [39]:
list=[[2,[1,2]],
      [2,[1,2]],
      [3,[1,2,3]],
      [3,[1,2]]]


df=spark.createDataFrame(list,['timestamp','vars'])

df.show()

+---------+---------+
timestamp| vars|
+---------+---------+
 2| [1, 2]|
 2| [1, 2]|
 3|[1, 2, 3]|
 3| [1, 2]|
+---------+---------+

In [40]:
w=Window().partitionBy("timestamp").orderBy(F.col("timestamp"))

df.withColumn("vars", F.flatten(F.collect_list("vars").over(w)))\
  .withColumn("num", F.row_number().over(w)).filter(F.col("num")==1).drop("num")\
  .show(truncate=False)


+---------+---------------+
timestamp|vars |
+---------+---------------+
3 |[1, 2, 3, 1, 2]|
2 |[1, 2, 1, 2] |
+---------+---------------+

In [41]:
df.show()


w=Window().partitionBy("timestamp").orderBy(F.col("timestamp"))
df.withColumn("vars", F.flatten((F.collect_list("vars").over(w))))\
  .withColumn("num", F.row_number().over(w)).filter(F.col("num")==1)\
  .drop("num").orderBy("timestamp")\
  .show()


+---------+---------+
timestamp| vars|
+---------+---------+
 2| [1, 2]|
 2| [1, 2]|
 3|[1, 2, 3]|
 3| [1, 2]|
+---------+---------+

+---------+---------------+
timestamp| vars|
+---------+---------------+
 2| [1, 2, 1, 2]|
 3|[1, 2, 3, 1, 2]|
+---------+---------------+

In [42]:
from pyspark.sql import functions as F
list=[[['one', 'two', 'three'],['one', 'two', 'three']],
      [['one', 'two', 'three'],['one', 'three', 'two']],
      [['one', 'two', 'three'],['one', 'two', 'three', 'three']],
      [['one', 'two', 'three'],['one', 'two', 'three', 'four']],
      [['one', 'two', 'three'],['one', 'two', 'four']],
      [['one', 'two', 'three'],['one']]]
df=spark.createDataFrame(list,['L1','L2'])
df.show(truncate=False)

+-----------------+------------------------+
L1 |L2 |
+-----------------+------------------------+
[one, two, three]|[one, two, three] |
[one, two, three]|[one, three, two] |
[one, two, three]|[one, two, three, three]|
[one, two, three]|[one, two, three, four] |
[one, two, three]|[one, two, four] |
[one, two, three]|[one] |
+-----------------+------------------------+

In [43]:
df.withColumn("result", F.concat(F.array_except("L1","L2"),F.array_except("L2","L1")))\
  .withColumn("Boolean", F.when(F.size("result")==0,F.lit(True)).otherwise(F.lit(False))).show(truncate=False)

+-----------------+------------------------+-------------+-------+
L1 |L2 |result |Boolean|
+-----------------+------------------------+-------------+-------+
[one, two, three]|[one, two, three] |[] |true |
[one, two, three]|[one, three, two] |[] |true |
[one, two, three]|[one, two, three, three]|[] |true |
[one, two, three]|[one, two, three, four] |[four] |false |
[one, two, three]|[one, two, four] |[three, four]|false |
[one, two, three]|[one] |[two, three] |false |
+-----------------+------------------------+-------------+-------+

In [44]:
df.withColumn("result", F.expr("""transform(L1, x-> x = array(struct(x)))""")).show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o1193.withColumn.
: org.apache.spark.sql.AnalysisException: cannot resolve '(namedlambdavariable() = array(named_struct('x', namedlambdavariable())))' due to data type mismatch: differing types in '(namedlambdavariable() = array(named_struct('x', namedlambdavariable())))' (string and array<struct<x:string>>).; line 1 pos 18;
'Project [L1#12898, L2#12899, transform(L1#12898, lambdafunction((lambda x#12924 = array(named_struct(x, lambda x#12924))), lambda x#12924, false)) AS result#12923]
+- LogicalRDD [L1#12898, L2#12899], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:126)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:302)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:105)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$cat

In [45]:
   L1                               L2
['one', 'two', 'three'] == ['one', 'two', 'three'] :  true
['one', 'two', 'three'] == ['one', 'three', 'two'] :  true
['one', 'two', 'three'] == ['one', 'two', 'three', 'three'] :  false, L1:'three'
['one', 'two', 'three'] == ['one', 'two', 'three', 'four'] :  false, L1:'four'
['one', 'two', 'three'] == ['one', 'two', 'four'] :  false, L1:'four', L2:'three'
['one', 'two', 'three'] == ['one'] :  false, L2:'two','three'

In [46]:
def getvowelcount(vowels,sentence):
     vowel=[i for i in vowels]
     new=[]
     for i in range(len(sentence)):
         new.append([i for i in sentence[i]])
     result=[]


     for i in range(len(new)):
        count=0 
        for j in range(len(vowel)):
           if vowel[j] in new[i]:
              count+=1
        result.append((''.join(new[i]),count))
        
     return result
        

getvowelcount('aeiou',['vikram', 'is', 'best', 'person'])

Out[102]: [('vikram', 2), ('is', 1), ('best', 1), ('person', 2)]

In [47]:
new 

Out[94]: [['v', 'i', 'k', 'r', 'a', 'm'],
 ['i', 's'],
 ['b', 'e', 's', 't'],
 ['p', 'e', 'r', 's', 'o', 'n']]

In [48]:
vowel

Out[95]: ['a', 'e', 'i', 'o', 'u']